# Прогнозирование цен потребительского ритейла по тестовой выборке на основе глубоких нейронных сетей
__Выполнил:__ *Домченко Максим*

__Студент группы:__ *РИМ-130962*

In [17]:
# ----------- ШАГ 1.1: Скачивание датасета -----------
# 📦 Цель: загрузить данные из Kaggle

# Для локального запуска загрузите kaggle.json в ~/.kaggle/kaggle.json и дайте права:
# !chmod 600 ~/.kaggle/kaggle.json

# Для запуска в colab

# from google.colab import files
# uploaded = files.upload('/root/.kaggle')
# !chmod 600 /root/.kaggle/kaggle.json

!kaggle competitions download -p ../data -c m5-forecasting-accuracy --force

  0%|                                               | 0.00/45.8M [00:00<?, ?B/s]
100%|██████████████████████████████████████| 45.8M/45.8M [00:00<00:00, 2.63GB/s]


In [18]:
# ----------- ШАГ 1.2: Распаковка архива -----------
# 📁 Цель: разархивировать m5-forecasting-accuracy.zip

import zipfile, os

zip_path = "../data/m5-forecasting-accuracy.zip"
extract_dir = "../data"

os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("✅ Архив распакован в:", extract_dir)

✅ Архив распакован в: ../data


In [19]:
# ----------- Шаг 2.1: Загрузка данных ----------- #
# 📥 Цель: считать три ключевых файла и сразу снизить объём памяти

import pandas as pd
import numpy as np
import os

# Типы для категориальных колонок в sales
dtype_sales = {col: 'category' for col in ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']}

# Загружаем данные
print("📥 Загружаем данные...")
sales = pd.read_csv(f"{extract_dir}/sales_train_validation.csv", dtype=dtype_sales)
calendar = pd.read_csv(f"{extract_dir}/calendar.csv", parse_dates=['date'])
prices = pd.read_csv(f"{extract_dir}/sell_prices.csv")

print(f"✅ Форматы считаны. Размеры:")
print(f"sales    : {sales.shape}")
print(f"calendar : {calendar.shape}")
print(f"prices   : {prices.shape}")

📥 Загружаем данные...
✅ Форматы считаны. Размеры:
sales    : (30490, 1919)
calendar : (1969, 14)
prices   : (6841121, 4)


In [20]:
# ----------- Шаг 2.2: Оптимизация числовых колонок ----------- #
# 📦 Цель: привести числовые столбцы к оптимальным типам и посчитать экономию памяти

def mem_usage(df):
    return round(df.memory_usage(deep=True).sum() / 1024**2, 1)

mem_before = {
    'sales': mem_usage(sales),
    'calendar': mem_usage(calendar),
    'prices': mem_usage(prices)
}

# sales: целые числа
sales_int = sales.select_dtypes(include='int').columns
sales[sales_int] = sales[sales_int].astype(np.uint16)

# calendar: float32 и int16
calendar_float = calendar.select_dtypes(include='float').columns
calendar[calendar_float] = calendar[calendar_float].astype(np.float32)

calendar_int = calendar.select_dtypes(include='int').columns
calendar[calendar_int] = calendar[calendar_int].astype(np.int16)

# prices: int и float
prices['wm_yr_wk'] = prices['wm_yr_wk'].astype(np.int16)
prices['sell_price'] = prices['sell_price'].astype(np.float32)

mem_after = {
    'sales': mem_usage(sales),
    'calendar': mem_usage(calendar),
    'prices': mem_usage(prices)
}

print("📊 Оптимизация памяти:")
for name in mem_before:
    before = mem_before[name]
    after = mem_after[name]
    delta = before - after
    perc = delta / before * 100 if before > 0 else 0
    print(f"▪ {name:8}: {before:>6} ➜ {after:>6} MiB ({perc:+.1f}% экономия)")

📊 Оптимизация памяти:
▪ sales   :  448.7 ➜  115.0 MiB (+74.4% экономия)
▪ calendar:    0.6 ➜    0.5 MiB (+16.7% экономия)
▪ prices  :  853.1 ➜  787.9 MiB (+7.6% экономия)


In [21]:
# ----------- Шаг 2.3: Оптимизация категориальных колонок ----------- #
# 🏷 Цель: перевести строковые признаки в category и сократить память

cat_cols_sales = ['id', 'item_id', 'dept_id', 'store_id', 'cat_id', 'state_id']
cat_cols_calendar = ['weekday', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
cat_cols_prices = ['item_id', 'store_id']

mem_before = {
    'sales': mem_usage(sales),
    'calendar': mem_usage(calendar),
    'prices': mem_usage(prices)
}

for col in cat_cols_sales:
    sales[col] = sales[col].astype('category')

for col in cat_cols_calendar:
    if col in calendar.columns:
        calendar[col] = calendar[col].astype('category')

for col in cat_cols_prices:
    prices[col] = prices[col].astype('category')

mem_after = {
    'sales': mem_usage(sales),
    'calendar': mem_usage(calendar),
    'prices': mem_usage(prices)
}

print("🏷 Оптимизация категориальных признаков:")
for name in mem_before:
    before = mem_before[name]
    after = mem_after[name]
    delta = before - after
    perc = delta / before * 100 if before > 0 else 0
    print(f"▪ {name:8}: {before:>6} ➜ {after:>6} MiB ({perc:+.1f}% экономия)")


🏷 Оптимизация категориальных признаков:
▪ sales   :  115.0 ➜  115.0 MiB (+0.0% экономия)
▪ calendar:    0.5 ➜    0.2 MiB (+60.0% экономия)
▪ prices  :  787.9 ➜   59.0 MiB (+92.5% экономия)


In [22]:
# ---------- Шаг 2.4: Проверка целостности данных ---------- #
# 🧪 Цель: убедиться, что данные остались корректными после оптимизации

print("🔍 Проверка целостности:")

# Проверяем размеры
print(f"- Размеры sales:     {sales.shape}")
print(f"- Размеры calendar:  {calendar.shape}")
print(f"- Размеры prices:    {prices.shape}")

# Проверяем базовую статистику
print("\n🧪 Примеры статистики:")
print(f"  Сумма продаж       : {sales.iloc[:, 6:].sum().sum():,.0f}")
print(f"  Уникальных дат     : {calendar['date'].nunique()}")
print(f"  Цены: min={prices['sell_price'].min()}, max={prices['sell_price'].max()}")

# Проверка типов
print("\n📋 Типы данных:")
print("sales:")
print(sales.dtypes.value_counts())

print("\ncalendar:")
print(calendar.dtypes.value_counts())

print("\nprices:")
print(prices.dtypes.value_counts())


🔍 Проверка целостности:
- Размеры sales:     (30490, 1919)
- Размеры calendar:  (1969, 14)
- Размеры prices:    (6841121, 4)

🧪 Примеры статистики:
  Сумма продаж       : 65,695,409
  Уникальных дат     : 1969
  Цены: min=0.009999999776482582, max=107.31999969482422

📋 Типы данных:
sales:
uint16      1913
category       1
category       1
category       1
category       1
category       1
category       1
Name: count, dtype: int64

calendar:
int16             7
datetime64[ns]    1
category          1
object            1
category          1
category          1
category          1
category          1
Name: count, dtype: int64

prices:
category    1
category    1
int16       1
float32     1
Name: count, dtype: int64


In [23]:
# ----------- Шаг 3.1: Приводим продажи к long-формату ----------- #
id_cols = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
product_info = sales[id_cols].copy()

# Удаляем метаинфо, оставляем только продажи
sales_long = sales.drop(columns=id_cols)

# Преобразуем в long-формат, id переименуем в row_id
sales_long = sales_long.reset_index().melt(
    id_vars='index',
    var_name='d',
    value_name='sales'
).rename(columns={'index': 'row_id'})

print("✅ Long-формат готов:", sales_long.shape)
display(sales_long.sample(5))


✅ Long-формат готов: (58327370, 3)


,row_id,d,sales
44404723,11283,d_1457,0
46347990,3190,d_1521,0
45393976,24856,d_1489,1
12320107,2147,d_405,1
42210080,11920,d_1385,1


In [24]:
# ----------- Шаг 3.2: Объединяем календарь, товары, цены ----------- #

# Добавим календарные признаки по "d"
sales_long = sales_long.merge(calendar, on='d', how='left')

# Добавим товарные признаки по row_id → id
sales_long = sales_long.merge(product_info, left_on='row_id', right_index=True, how='left')

# Добавим цену по ключам
sales_long = sales_long.merge(
    prices[['store_id', 'item_id', 'wm_yr_wk', 'sell_price']],
    on=['store_id', 'item_id', 'wm_yr_wk'],
    how='left'
)

# Проверка
print("✅ Данные объединены. Размер:", sales_long.shape)
display(sales_long.sample(5))

# Проверим пропуски
missing_cols = ['item_id', 'dept_id', 'cat_id', 'store_id', 'sell_price']
print("\n🔍 Доля пропусков:")
print(sales_long[missing_cols].isna().mean().sort_values(ascending=False))


✅ Данные объединены. Размер: (58327370, 23)


,row_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,...,snap_CA,snap_TX,snap_WI,id,item_id,dept_id,cat_id,store_id,state_id,sell_price
55440273,9453,d_1819,0,2016-01-21,11551,Thursday,6,1,2016,NaN,...,0,0,0,HOBBIES_1_315_CA_4_validation,HOBBIES_1_315,HOBBIES_1,HOBBIES,CA_4,CA,4.97
39084289,26599,d_1282,2,2014-08-02,11427,Saturday,1,8,2014,NaN,...,1,0,1,FOODS_2_381_WI_2_validation,FOODS_2_381,FOODS_2,FOODS,WI_2,WI,2.75
5239274,25484,d_172,0,2011-07-19,11125,Tuesday,4,7,2011,NaN,...,0,0,0,HOUSEHOLD_1_537_WI_2_validation,HOUSEHOLD_1_537,HOUSEHOLD_1,HOUSEHOLD,WI_2,WI,15.97
34643247,6607,d_1137,0,2014-03-10,11406,Monday,3,3,2014,NaN,...,1,0,0,HOBBIES_2_094_CA_3_validation,HOBBIES_2_094,HOBBIES_2,HOBBIES,CA_3,CA,2.47
23594219,25449,d_774,0,2013-03-12,11307,Tuesday,4,3,2013,NaN,...,0,1,1,HOUSEHOLD_1_502_WI_2_validation,HOUSEHOLD_1_502,HOUSEHOLD_1,HOUSEHOLD,WI_2,WI,NaN



🔍 Доля пропусков:
sell_price    0.210869
item_id       0.000000
dept_id       0.000000
cat_id        0.000000
store_id      0.000000
dtype: float64


In [25]:
# ----------- Шаг 3.3: Обработка пропусков в ценах ----------- #
sales_long['has_price'] = sales_long['sell_price'].notna().astype('int')

print("✅ Добавлен флаг наличия цены. Пример:")
display(sales_long[['sell_price', 'has_price']].sample(5))


✅ Добавлен флаг наличия цены. Пример:


,sell_price,has_price
14871790,2.38,1
20541633,2.47,1
18435186,7.97,1
56460759,4.44,1
4622056,NaN,0


In [26]:
# ----------- Шаг 4.1: Временные признаки ----------- #
# 🧠 Цель: извлечь day, weekday, month, quarter и другие признаки из даты

# Убедимся, что date — это datetime
sales_long['date'] = pd.to_datetime(sales_long['date'])

# Временные признаки
sales_long['day']        = sales_long['date'].dt.day
sales_long['weekday']    = sales_long['date'].dt.day_name()
sales_long['weekday_num']= sales_long['date'].dt.weekday  # Monday = 0
sales_long['is_weekend'] = sales_long['weekday_num'].isin([5, 6]).astype(int)
sales_long['month']      = sales_long['date'].dt.month
sales_long['quarter']    = sales_long['date'].dt.quarter
sales_long['year']       = sales_long['date'].dt.year
sales_long['week']       = sales_long['date'].dt.isocalendar().week.astype('int16')

print("✅ Временные признаки добавлены. Пример:")
display(sales_long[['date', 'weekday', 'is_weekend', 'month', 'quarter', 'week']].sample(5))


✅ Временные признаки добавлены. Пример:


,date,weekday,is_weekend,month,quarter,week
52134616,2015-10-04,Sunday,1,10,4,40
22967978,2013-02-20,Wednesday,0,2,1,8
50177204,2015-08-01,Saturday,1,8,3,31
13799850,2012-04-25,Wednesday,0,4,2,17
13252294,2012-04-07,Saturday,1,4,2,14


In [27]:
# ----------- Шаг 4.2: Признаки событий и логарифм цены ----------- #
# 🎯 Цель: добавить флаг события и log-преобразование цены

# Преобразуем цену (log1p — безопасно для нулей)
sales_long['log_price'] = np.log1p(sales_long['sell_price'])

# Признак наличия события
sales_long['is_event'] = (
    sales_long['event_name_1'].notna() | sales_long['event_name_2'].notna()
).astype('int')

print("✅ Добавлены признаки is_event и log_price. Пример:")
display(sales_long[['sell_price', 'log_price', 'event_name_1', 'event_name_2', 'is_event']].sample(5))


✅ Добавлены признаки is_event и log_price. Пример:


,sell_price,log_price,event_name_1,event_name_2,is_event
8125590,NaN,NaN,NaN,NaN,0
43361575,2.00,1.098612,NaN,NaN,0
13782250,4.68,1.736951,NaN,NaN,0
18816236,NaN,NaN,NaN,NaN,0
27632869,1.98,1.091923,NaN,NaN,0


In [28]:
# Доля пропусков в sell_price и log_price
print("🧮 Доля пропусков:")
print(sales_long[['sell_price', 'log_price']].isna().mean())

# Распределение флага событий
print("\n🧾 Распределение is_event:")
print(sales_long['is_event'].value_counts(normalize=True))

# Примеры с is_event == 1
print("\n🔍 Примеры строк с событиями:")
display(sales_long[sales_long['is_event'] == 1][['date', 'event_name_1', 'event_name_2', 'is_event']].sample(5))


🧮 Доля пропусков:
sell_price    0.210869
log_price     0.210869
dtype: float64

🧾 Распределение is_event:
is_event
0    0.919498
1    0.080502
Name: proportion, dtype: float64

🔍 Примеры строк с событиями:


,date,event_name_1,event_name_2,is_event
2951616,2011-05-05,Cinco De Mayo,NaN,1
35903788,2014-04-20,Easter,OrthodoxEaster,1
34498953,2014-03-05,LentStart,NaN,1
54286434,2015-12-14,Chanukah End,NaN,1
30156682,2013-10-14,ColumbusDay,NaN,1


In [ ]:
# ----------- Шаг 5.1: Лаги и скользящие окна ----------- #
# 🎯 Цель: добавить признаки отставания и сглаживания по продажам

group_cols = ['id']

# Добавим лаги (1, 7, 14, 28 дней)
for lag in [1, 7, 14, 28]:
    sales_long[f'lag_{lag}'] = (
        sales_long.groupby(group_cols, observed=True)['sales']
        .shift(lag)
    )

# Добавим скользящие средние (7, 14, 28 дней)
for window in [7, 14, 28]:
    sales_long[f'rolling_mean_{window}'] = (
        sales_long.groupby(group_cols, observed=True)['sales']
        .transform(lambda x: x.shift(1).rolling(window).mean())
    )

# ✅ Валидация: проверим наличие признаков и отсутствие NaN в середине
print("📊 Примеры лагов и скользящих средних:")
cols_to_check = [f'lag_{l}' for l in [1, 7, 14, 28]] + [f'rolling_mean_{w}' for w in [7, 14, 28]]
display(sales_long[cols_to_check].dropna().sample(5))

# 🧪 Проверка: сколько процентов NaN — допустимы только в начале рядов
print("\n🔎 Доля пропусков:")
print(sales_long[cols_to_check].isna().mean().sort_values(ascending=False))


In [ ]:
# ---------- Шаг 5.2: Статистики по товарам и магазинам ---------- #
# 🧮 Цель: добавить mean, median, std, min, max по товарам и магазинам

agg_funcs = ['mean', 'median', 'std', 'min', 'max']
group_targets = {
    'id': 'id',
    'store': 'store_id'
}

for group_name, group_col in group_targets.items():
    for func in agg_funcs:
        col_name = f'{group_name}_sales_{func}'
        sales_long[col_name] = (
            sales_long.groupby(group_col, observed=True)['sales']
            .transform(func)
        )

print("✅ Добавлены агрегаты по товарам и магазинам.")
cols_added = [f'{g}_sales_{f}' for g in group_targets for f in agg_funcs]
display(sales_long[cols_added].dropna().sample(5))

# Проверка доли пропусков
print("\n🧮 Пропуски в новых признаках:")
print(sales_long[cols_added].isna().mean().sort_values(ascending=False))


In [ ]:
# ---------- Шаг 5.3: EWMA-тренды по продажам ---------- #
# 🎯 Цель: добавить экспоненциально взвешенное среднее (EWMA) по продажам

ewma_spans = [7, 14, 28]
for span in ewma_spans:
    col_name = f'ewma_{span}'
    sales_long[col_name] = (
        sales_long.groupby('id', observed=True)['sales']
        .transform(lambda x: x.shift(1).ewm(span=span).mean())
    )

print("✅ EWMA-тренды добавлены:")
display(sales_long[[f'ewma_{s}' for s in ewma_spans]].dropna().sample(5))

# Проверка доли пропусков — допустимы в начале ряда
print("\n📉 Пропуски в EWMA:")
print(sales_long[[f'ewma_{s}' for s in ewma_spans]].isna().mean().sort_values(ascending=False))


In [ ]:
# ----------- Шаг 5.4: STL-декомпозиция по цене (ускоренная) ----------- #
# 🧠 Цель: выделить тренд и сезонность по `sell_price`

from statsmodels.tsa.seasonal import STL
import numpy as np

# Параметры
season_length = 28

# Группируем по уникальным товарам
def apply_stl(group):
    """Возвращает тренд и сезонность, если ряд достаточно длинный"""
    if group['sell_price'].notna().sum() >= 2 * season_length:
        result = STL(group['sell_price'].interpolate(), period=season_length, robust=True).fit()
        return pd.DataFrame({
            'stl_trend': result.trend,
            'stl_seasonal': result.seasonal
        }, index=group.index)
    else:
        return pd.DataFrame({
            'stl_trend': np.nan,
            'stl_seasonal': np.nan
        }, index=group.index)

# Применяем STL
print("🚀 Запускаем STL-декомпозицию по цене:")
stl_result = sales_long.groupby('id', observed=True).apply(apply_stl).reset_index(level=0, drop=True)

# Присоединяем к sales_long
sales_long = sales_long.join(stl_result)

# Проверка
print("✅ STL-декомпозиция завершена.")
display(sales_long[['sell_price', 'stl_trend', 'stl_seasonal']].dropna().sample(5))

print("\n🧪 Пропуски после STL:")
print(sales_long[['stl_trend', 'stl_seasonal']].isna().mean())